In [1]:
import torch
# 创建一个5×3的随机矩阵并显示
x = torch.rand(5,3)
x


 0.6439  0.6374  0.8579
 0.4668  0.0008  0.1469
 0.0853  0.2430  0.5965
 0.0163  0.7560  0.4052
 0.3702  0.6134  0.4598
[torch.FloatTensor of size 5x3]

# tensor的运算


In [2]:
y = torch.ones(5,3)
# 创建一个5×3的全是1矩阵并显示
y



 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
[torch.FloatTensor of size 5x3]

In [3]:
# 计算两个矩阵相加(注意尺寸要求一样)
z = x + y
z



 1.6439  1.6374  1.8579
 1.4668  1.0008  1.1469
 1.0853  1.2430  1.5965
 1.0163  1.7560  1.4052
 1.3702  1.6134  1.4598
[torch.FloatTensor of size 5x3]

In [5]:
# 矩阵乘法，矩阵转置
q = x.mm(y.t())
q


 2.1392  2.1392  2.1392  2.1392  2.1392
 0.6145  0.6145  0.6145  0.6145  0.6145
 0.9249  0.9249  0.9249  0.9249  0.9249
 1.1774  1.1774  1.1774  1.1774  1.1774
 1.4433  1.4433  1.4433  1.4433  1.4433
[torch.FloatTensor of size 5x5]

所有Numpy上面关于ndarray的运算全部可以应用于tensor

有关tensor的运算参考 http://pytorch.org/docs/master/tensors.html

从numpy到tensor的转换：torch.from_numpy(a)

从tensor到numpy的转换：a.numpy()

tensor与numpy的最大不同：tensor可以在GPU上运算

转到gpu上运算(x.cpu()转成cpu)

In [6]:
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    print(x + y)


 1.6439  1.6374  1.8579
 1.4668  1.0008  1.1469
 1.0853  1.2430  1.5965
 1.0163  1.7560  1.4052
 1.3702  1.6134  1.4598
[torch.cuda.FloatTensor of size 5x3 (GPU 0)]



# Numpy桥

将Torch的Tensor和numpy的array相互转换。注意Torch的Tensor和numpy的array会共享他们的存储空间，修改一个会导致另外的一个也被修改。

In [14]:
# 此处演示tensor和numpy数据结构的相互转换
a = torch.ones(5)
b = a.numpy()
print(a)
print(b)


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]

[1. 1. 1. 1. 1.]


In [10]:
# 此处演示当修改numpy数组之后,与之相关联的tensor也会相应的被修改
a.add_(1)
print(a)
print(b)


 2
 2
 2
 2
 2
[torch.FloatTensor of size 5]

[2. 2. 2. 2. 2.]


In [16]:
# 将numpy的Array转换为torch的Tensor
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
print(a)
print(b)
np.add(a,1,out=a)
print(a)
print(b)

[1. 1. 1. 1. 1.]

 1
 1
 1
 1
 1
[torch.DoubleTensor of size 5]

[2. 2. 2. 2. 2.]

 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]



In [18]:
# 另外除了CharTensor之外，所有的tensor都可以在CPU运算和GPU预算之间相互转换
# 使用CUDA函数来将Tensor移动到GPU上
# 当CUDA可用时会进行GPU的运算
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    print(x + y)


 1.6439  1.6374  1.8579
 1.4668  1.0008  1.1469
 1.0853  1.2430  1.5965
 1.0163  1.7560  1.4052
 1.3702  1.6134  1.4598
[torch.cuda.FloatTensor of size 5x3 (GPU 0)]



# 动态计算图(Dynamic Computation Graph)



# 是pytorch的最主要特征
# 让计算模型更灵活，复杂
# 让反向传播算法随时进行

##自动微分变量
gradient梯度，传播的就是梯度

定义一个自动微分变量

In [20]:
from torch.autograd import Variable
# Variable:自动微分变量
x = Variable(torch.ones(2,2),requires_grad = True)
#把一个2*2的张量转变成微分的变量（添加节点，构造计算图）
x

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

In [34]:
y = x + 2
print(y)
# 因为y是通过一个操作创建的,所以它有grad_fn,而x是由用户创建,所以它的grad_fn为None.
print(y.grad_fn)
print(x.grad_fn)

z = torch.mean(y*y)
print(z)

z.data


Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

None
Variable containing:
 9
[torch.FloatTensor of size 1]




 9
[torch.FloatTensor of size 1]

In [37]:
z = y * y * 3
out = z.mean()
print(z,out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



# 梯度(Gradients)
现在我们来执行反向传播,out.backward()相当于执行out.backward(torch.Tensor([1.0]))

In [38]:
out.backward()


In [39]:
# 输出out对x的梯度d(out)/dx:
print(x.grad)

Variable containing:
 18  18
 18  18
[torch.FloatTensor of size 2x2]



更疯狂的函数依赖



In [41]:
s = Variable(torch.FloatTensor([[0.01,0.02]]),requires_grad = True)
x = Variable(torch.ones(2,2),requires_grad = True)
for i in range(10):
    s = s.mm(x) # 矩阵乘法
z = torch.mean(s)
print(z)

Variable containing:
 15.3600
[torch.FloatTensor of size 1]



In [42]:
# backward()求导计算
z.backward()
print(x.grad)
print(s.grad)

Variable containing:
 37.1200  37.1200
 39.6800  39.6800
[torch.FloatTensor of size 2x2]

None


# 神经网络
可以使用torch.nn包来构建神经网络.

你已知道autograd包,nn包依赖autograd包来定义模型并求导.一个nn.Module包含各个层和一个faward(input)方法,该方法返回output.

例如,我们来看一下下面这个分类数字图像的网络.

神经网络的典型训练过程如下: 
1. 定义神经网络模型,它有一些可学习的参数(或者权重); 
2. 在数据集上迭代; 
3. 通过神经网络处理输入; 
4. 计算损失(输出结果和正确值的差距大小) 
5. 将梯度反向传播会网络的参数; 
6. 更新网络的参数,主要使用如下简单的更新原则

weight = weight - learning_rate * gradient 



# 我们先定义一个网络


In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 1 input image channel, 6 output channels, 5*5 square convolution
        # kernel
        
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # an affine operation :y = Wx + b
        self.fc1 = nn.Linear(16 *5 * 5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
        
    def forward(self,x):
        # max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # If size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    
    def num_flat_features(self,x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


仅仅需要定义一个forward函数就可以了，backward会自动地生成。

你可以在forward函数中使用所有的Tensor中的操作。

模型中可学习的参数会由net.parameters()返回。



In [45]:
params = list(net.parameters())
print(len(params))
for param in params:
    print(param.size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


forward的输入和输出都是autograd.Variable.注意:这个网络(LeNet)期望的输入大小是32*32.如果使用MNIST数据集来训练这个网络,请把图片大小重新调整到32*32.

In [46]:
input = Variable(torch.rand(1,1,32,32))
out = net(input)
print(out)


Variable containing:
-0.0524 -0.0706 -0.0473  0.1264 -0.1027 -0.0257 -0.1180  0.0642 -0.0487  0.0590
[torch.FloatTensor of size 1x10]



In [47]:
# 将所有参数的梯度缓存清零,然后进行随机梯度的的反向传播.
net.zero_grad()
out.backward(torch.randn(1,10))


注意: torch.nn 只接受小批量的数据

整个torch.nn包只接受那种小批量样本的数据，而非单个样本。 例如，nn.Conv2d能够结构一个四维的TensornSamples x nChannels x Height x Width。

如果你拿的是单个样本，使用input.unsqueeze(0)来加一个假维度就可以了。

